In [ ]:
%cd ..
%pwd

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
# Now we can override it and set it to "AI Assistant"
from langchain.prompts.prompt import PromptTemplate

template = """
        Tu es un assistant vocal de conduite. Grâce à une caméra intégrée dans la voiture, le taux d'alcoolémie du conducteur a été mesuré.
        Sache qu'un verre standard d'alccol équivault à 0.2g/L. Si le taux est supérieur à 0.5g/L, tu dois conseiller au conducteur de ne pas conduire. 
        Reste concis très concis dans tes réponses.
Conversation actuelle:
{history}
Utilisateur: {input}
Assistant Vocal:"""

SYSTEM_PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


llm = ChatGroq(groq_api_key=groq_api_key, model="mixtral-8x7b-32768")
conversation = ConversationChain(
    prompt=SYSTEM_PROMPT,
    llm=llm,
    verbose=False,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant"),
)


In [ ]:
inputs = [
    "Bonjour VigilAuto, on rentre à la maison !",
    "Oh non, je n'ai pas beaucoup bu ce soir, seulement un verre de vin.",
    "Combien ai-je bu déjà ?",
    "Merci Vigil'Auto, on est partis !",
]
for user_input in inputs:
    print(f"User: {user_input}")
    print(f"AI Assistant: {conversation.predict(input=user_input)}")

In [ ]:
from VigilAuto.entity.config_entity import LLMConfig
class LLMChat:
    def __init__(self, config: LLMConfig):
        template = """
        Tu es un assistant vocal de conduite. Grâce à une caméra intégrée dans la voiture, le taux d'alcoolémie du conducteur a été mesuré.
        Sache qu'un verre standard d'alccol équivault à 0.2g/L. Si le taux est supérieur à 0.5g/L, tu dois conseiller au conducteur de ne pas conduire. 
        Reste concis très concis dans tes réponses.
        Conversation actuelle:
        {history}
        Utilisateur: {input}
        Assistant Vocal:"""
        self.config = config
        self.llm = ChatGroq(groq_api_key=groq_api_key, model=self.config.model_name)
        self.conversation = ConversationChain(
            prompt=PromptTemplate(input_variables=["history", "input"], template=template),
            llm=self.llm,
            verbose=False,
            memory=ConversationBufferMemory(ai_prefix="Assistant Vocal", human_prefix="Conducteur"),
        )

    def predict(self, input: str):
        return self.conversation.predict(input=input)

In [ ]:
from VigilAuto.config.configuration import ConfigurationManager
config_manager = ConfigurationManager()
llm_config = config_manager.get_llm_config()
llm = LLMChat(config=llm_config)

In [ ]:
llm.predict("Bonjour VigilAuto, je m'appelle Jules et je suis le conducteur de la voiture.")

In [ ]:
llm.predict("Quel est mon nom ?")